<a href="https://colab.research.google.com/github/vksriharsha/Aminobert_ESM2_Comparative_Study/blob/main/src/Aminobert_ESM2_Comparison.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install torch
!pip install fair-esm
!pip install fair-esm[esmfold]
!pip install Bio
!pip install biopython
# OpenFold and its remaining dependency
!pip install 'dllogger @ git+https://github.com/NVIDIA/dllogger.git'
!pip install 'openfold @ git+https://github.com/aqlaboratory/openfold.git@4b41059694619831a7db195b7e0988fc4ff3a307'

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 93 kB 1.2 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 77 kB 3.5 MB/s 
     |████████████████████████████████| 798 kB 32.5 MB/s 
     |████████████████████████████████| 3.0 MB 32.5 MB/s 
     |████████████████████████████████| 41 kB 457 kB/s 
     |████████████████████████████████| 79 kB 8.6 MB/s 
     |████████████████████████████████| 510 kB 59.8 MB/s 
     |████████████████████████████████| 54 kB 2.9 MB/s 
     |████████████████████████████████| 145 kB 49.2 MB/s 
     |████████████████████████████████| 15.2 MB 45.4 MB/s 
     |████████████████████████████████| 117 kB 56.1 MB/s 
     |████████████████████████████████| 529 kB 61.3 MB/s 
     |██████████

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 272 kB 5.3 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/NVIDIA/dllogger.git to /tmp/pip-install-k7x71tbb/dllogger_e61db787548d48fb8cc7e02bb9b0daf1
  Running command git clone -q https://github.com/NVIDIA/dllogger.git /tmp/pip-install-k7x71tbb/dllogger_e61db787548d48fb8cc7e02bb9b0daf1
  Created wheel for dllogger: filename=DLLogger-1.0.0-py3-none-any.whl size=5670 sha256=8c20780013f8571e4842de2333bed295d5b47d893498415b5e5c9b741311d50d
  Stored in directory: /tmp/pip-ephem-wheel-cache-oa4yzoh7/wheels/db/ba/1b/87515aba93adffc7caccc21c0e93f80b70a857188790ce0436
Successfully built dllogger
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-whee

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import glob
import torch
import esm
import os
import re
from ctypes import sizeof
from collections import OrderedDict 
import sys
import shutil
import numpy as np
from tqdm import tqdm

# Pre-processing

## Set folder path of the PDB files

In [ ]:
# Folder Path
path = r"/content/drive/MyDrive/PDB"

## Function to convert PDB to FASTA file

In [ ]:
def convertPDBtoFASTA(pdb_file):
    aa3to1={
       'ALA':'A', 'VAL':'V', 'PHE':'F', 'PRO':'P', 'MET':'M',
       'ILE':'I', 'LEU':'L', 'ASP':'D', 'GLU':'E', 'LYS':'K',
       'ARG':'R', 'SER':'S', 'THR':'T', 'TYR':'Y', 'HIS':'H',
       'CYS':'C', 'ASN':'N', 'GLN':'Q', 'TRP':'W', 'GLY':'G',
       'MSE':'M', 'UNK':'X',
    }
    
    ca_pattern=re.compile("^ATOM\s{2,6}\d{1,5}\s{2}CA\s[\sA]([A-Z]{3})\s([\s\w])|^HETATM\s{0,4}\d   {1,5}\s{2}CA\s[\sA](MSE)\s([\s\w])")

    filename=os.path.basename(pdb_file).split('.')[0]
    dirname=os.path.dirname(pdb_file)
    chain_dict=dict()
    chain_list=[]

    fp=open(pdb_file,'rU')
    for line in fp.read().splitlines():
        if line.startswith("ENDMDL"):
            break
        match_list=ca_pattern.findall(line)
        if match_list:
            resn=match_list[0][0]+match_list[0][2]
            chain=match_list[0][1]+match_list[0][3]
            if chain in chain_dict:
                chain_dict[chain]+=aa3to1[resn]
            else:
                chain_dict[chain]=aa3to1[resn]
                chain_list.append(chain)
    fp.close()

    for chain in chain_list:

      if not os.path.exists(dirname+"/output/"+filename+"/"):
        os.makedirs(os.path.dirname(dirname+"/output/"+filename+"/"))

      fasta_file = open(dirname+"/output/"+filename+"/"+filename+".fasta", "a")
      fasta_file.write('>%s:%s\n%s\n'%(filename,chain,chain_dict[chain]))
      fasta_file.close()
      shutil.copy(dirname+"/"+filename+".pdb", dirname+"/output/"+filename+"/"+filename+".pdb")


## Converting all PDB files to FASTA

In [ ]:
# Import Module
import os

# Change the directory
os.chdir(path)

# Read text File
def read_text_file(file_path):
	with open(file_path, 'r') as f:
		print(f.read())


# iterate through all file

print("Converting PDB files to FASTA files and writing to:"+ path + "/output" )
for file in tqdm(os.listdir()):
	# Check whether file is in text format or not
	if file.endswith(".pdb"):
		file_path = f"{path}/{file}"
		convertPDBtoFASTA(file_path)
	

print("\n")


Converting PDB files to FASTA files and writing to:/content/drive/MyDrive/PDB/output


100%|██████████| 861/861 [04:37<00:00,  3.10it/s]

## Function to get all second sequences in FASTA files

In [ ]:
def get_second_line(file):
	with open(file, 'r') as f:
		return f.readlines()[1]


In [ ]:
path = path+"/output"

# Change the directory
os.chdir(path)

# Read text File
def read_text_file(file_path):
	with open(file_path, 'r') as f:
		print(f.read())


# iterate through all file
ff=[]
all_folders = os.listdir()

print("Extracting the second sequence in all fasta files...")
for folder in tqdm(all_folders):
	os.chdir(path+"/"+folder)
	for file in os.listdir():
		if file.endswith(".fasta"):
			file_path = f"{path}/{folder}/{file}"
			ff.append((file,get_second_line(file_path)))


Extracting the second sequence in all fasta files...


100%|██████████| 430/430 [00:00<00:00, 625.96it/s]


# ESM-2

## Load ESM-2 pretrained model

In [ ]:
model, alphabet = esm.pretrained.esm2_t33_650M_UR50D()
batch_converter = alphabet.get_batch_converter()

Downloading: "https://dl.fbaipublicfiles.com/fair-esm/models/esm2_t33_650M_UR50D.pt" to /root/.cache/torch/hub/checkpoints/esm2_t33_650M_UR50D.pt
Downloading: "https://dl.fbaipublicfiles.com/fair-esm/regression/esm2_t33_650M_UR50D-contact-regression.pt" to /root/.cache/torch/hub/checkpoints/esm2_t33_650M_UR50D-contact-regression.pt


## *** For Google Colab: Limiting number of proteins to 10

Due to memory constraints

In [ ]:
ff = ff[0:3]

ff

[('2a06_bundle.fasta',
  'WWNFGSLLGICLILQILTGLFLAIRYMHANGASMFFICLYMHVGRGLYTWNIGVILLLTVMATAFMGYFAFHFILPFIIMAIAMVHLIKDILGALLLILALMLLVLFALGGVLALAFSILILALIPLLSQCLFWALVADLLTLTWIGGITIGQLASVLYFLLILVLMPT\n'),
 ('2a65_bundle.fasta',
  'GLILAMAGNAVGLGNFLRFPMIPYIIAFLLVGIPLMWIEWAKILGVFGLWIPLVVAIYYVYIESWTLGFAIKFLVFAYIVFLITMFINVSILIRIERFAKIAMPTLFILAVFLVIRGVWIAAVGQIFFTLSLGFGAIITYLSGLTAATLNEKAEVILGGSILGFLWFFLLFFAGLTSSIAIMQPMIAFLAVLWTAAIVFFSAHLVMFLLDEMDFWAGTIGVVFFGLTELIIFFWIPRIYYYVMRYITPAFLAVLLVVWARWITRFYIIGLFLFLTFLVF\n'),
 ('2b2f_bundle.fasta',
  'ASTALVMLMVPGVGFFYAIALSFISLIITVLLWIFYGYSVSMYQMMFAAVTIAILTSAISSFILLSALWLTFVYAPFAHWLFAGGMVVHISSGFAALAVAMTLTLIGAALLWFGWFGFNGGSVVVTNTSAAVAGFVWMVIGWILGIVSGAIAGLAAITPAAKGAIVIGLVAGIVCYLAMDFRSLDAWAIHGIGGLWGSVAVGILLIAVASTTAYAFLVTLILAKAVDAAV\n')]

## Generate ESM-2 embeddings

In [ ]:
from math import e
print("Generating protein embeddings using ESM-2...")
for tuple in tqdm(ff):
  data = [tuple]
  batch_labels, batch_strs, batch_tokens = batch_converter(data)
  batch_lens = (batch_tokens != alphabet.padding_idx).sum(1)

  # Extract per-residue representations (on CPU)
  with torch.no_grad():
      results = model(batch_tokens, repr_layers=[33], return_contacts=True)
  token_representations = results["representations"][33]

  # Generate per-sequence representations via averaging
  # NOTE: token 0 is always a beginning-of-sequence token, so the first residue is token 1.
  sequence_representations = []
  for i, tokens_len in enumerate(batch_lens):
      sequence_representations.append(token_representations[i, 1 : tokens_len - 1].mean(0))
  
  # file = open(path+"/"+data[0][0].split(".")[0]+"/"+data[0][0].split(".")[0]+"_esm2.txt", "w+")
  if len(sequence_representations)< 2:
    # torch.save(sequence_representations[0], path+"/"+data[0][0].split(".")[0]+"/"+data[0][0].split(".")[0]+"_esm2.txt")
    np.savetxt(path+"/"+data[0][0].split(".")[0]+"/"+data[0][0].split(".")[0]+"_esm2.txt", torch.Tensor(sequence_representations[0]).numpy())
    # content = str(sequence_representations[0])
    # file.write(content)
    # file.close()
  else:
    print("more than one representations")
  # Saving the array in a text file
  # content = str(sequence_representations)
  # file.write(content)
  # file.close()


# # Look at the unsupervised self-attention map contact predictions
# import matplotlib.pyplot as plt
# for (_, seq), tokens_len, attention_contacts in zip(ff, batch_lens, results["contacts"]):
#     plt.matshow(attention_contacts[: tokens_len, : tokens_len])
#     plt.title(seq)
#     plt.show()

Generating protein embeddings using ESM-2...


100%|██████████| 3/3 [00:16<00:00,  5.61s/it]


In [ ]:
sequence_representations

[tensor([ 0.0435, -0.0686,  0.0389,  ...,  0.0573, -0.0584, -0.1091]),
 tensor([ 0.0514, -0.0141, -0.0357,  ...,  0.0535, -0.0585, -0.0356]),
 tensor([-0.0232, -0.0100,  0.0172,  ...,  0.0560, -0.0582, -0.0600])]